In [1]:
import pandas as pd

In [2]:
email_path = "../data/Email_Campaign_Details.csv"
# email_df = pd.read_csv(email_path, encoding='mac_roman', parse_dates=["Sent_date"])
email_df = pd.read_csv(email_path, encoding='mac_roman')
email_df

,Email_Campaign_id,contact_id,email_subject,is_sent,is_delivered,is_link_clicked,is_hard_bounce,is_bounce,is_opt_out,is_open,Sent_date,Campaign_type,Primary Topic
0,701200000006cIGAAY,00v2000000IZVYWAA5,Roadshow Houston (Advisor),1,1,0,0,0,0,1,2/7/2023,Events - In Person,/
1,701200000006cIGAAY,00v2000000IZW13AAH,Roadshow Houston (Advisor),1,1,0,0,0,0,1,2/7/2023,Events - In Person,/
2,701200000006cIGAAY,00v2000000IZUepAAH,Roadshow Houston (Advisor),1,1,0,0,0,0,0,2/7/2023,Events - In Person,/
3,701200000006cIGAAY,00v2000000IZWDdAAP,Roadshow Houston (Advisor),1,1,0,0,0,0,0,2/7/2023,Events - In Person,/
4,701200000006cIGAAY,00v2000000IZUdSAAX,Roadshow Houston (Advisor),1,1,0,0,0,0,0,2/7/2023,Events - In Person,/
...,...,...,...,...,...,...,...,...,...,...,...,...,...
128231,701D0000000NHUeIAO,00v2000000ZCFyUAAX,Meeting for Product fund Insights,1,1,0,0,0,0,0,6/22/2023,Digital event,Adhoc
128232,701D0000000NHUeIAO,00v2000000KrgKjAAJ,Meeting for Product fund Insights,1,1,0,0,0,0,0,6/22/2023,Digital event,Adhoc
128233,701D0000000NHUeIAO,00v2000000H3dwgAAB,Meeting for Product fund Insights,1,1,0,0,0,0,0,6/22/2023,Digital event,Adhoc
128234,701D0000000NHUeIAO,00v2000000Mj8xWAAR,Meeting for Product fund Insights,1,1,0,0,0,0,0,6/22/2023,Digital event,Adhoc


In [3]:
## Fix spelling error in the Primary Topic column

topic_map = {
    "Ad hoc": "Adhoc",
    "Asssets": "Asset",
    "--": "Undefined",
    "/": "Undefined"
}

email_df["Primary Topic"] = email_df["Primary Topic"] \
    .map(lambda x: x if x not in topic_map.keys() else topic_map[x])

## Fix some of the NULL Primary Topics


## Convert all the NULLs in Primary Topic to "Undefined"

email_df["Primary Topic"] = email_df["Primary Topic"].fillna("Undefined")


In [4]:
email_df["Primary Topic"].value_counts()

Primary Topic
ETF                34706
Undefined          16484
Adhoc              15799
Bonds              14321
Portfolio          13840
Market Insights    13226
Equities            6514
Insights            5326
HealthCare          4943
Asset               1156
Funds                814
Income               465
Investment           449
Retirement           187
Growth                 6
Name: count, dtype: int64

In [5]:
email_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128236 entries, 0 to 128235
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Email_Campaign_id  128236 non-null  object
 1   contact_id         128236 non-null  object
 2   email_subject      128236 non-null  object
 3   is_sent            128236 non-null  int64 
 4   is_delivered       128236 non-null  int64 
 5   is_link_clicked    128236 non-null  int64 
 6   is_hard_bounce     128236 non-null  int64 
 7   is_bounce          128236 non-null  int64 
 8   is_opt_out         128236 non-null  int64 
 9   is_open            128236 non-null  int64 
 10  Sent_date          128236 non-null  object
 11  Campaign_type      128236 non-null  object
 12  Primary Topic      128236 non-null  object
dtypes: int64(7), object(6)
memory usage: 12.7+ MB


In [6]:
## drop 3 rows where email is not opened but link is clicked

email_df.drop(email_df[(email_df["is_open"] == 0) & (email_df["is_link_clicked"] == 1)].index,
              inplace=True
             )


In [7]:
# join contact dataframe with email campaign data

contact_df = pd.read_csv("../cleaned_data/contact_details_cleaned.csv")

email_df = email_df.merge(contact_df,
                          how="inner",
                          left_on = "contact_id",
                          right_on = "Contact_id"
                         )
email_df.drop(["Contact_id"], axis=1, inplace=True)

In [8]:
clean_email_path = "../cleaned_data/email_campaign_cleaned.csv"
email_df.to_csv(clean_email_path, index=False)